In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import glob
import random
from data_loader import full_load_map, data_dir, load_map, Note
from concurrent.futures import ProcessPoolExecutor
import json
import shutil
import os
import traceback

In [2]:
tf.config.optimizer.set_jit('autoclustering')

In [3]:
random_seed = 1470258369

random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [4]:
def data_generator_multi_process(map_folders):
    map_folders = [map_folder.decode('UTF-8') for map_folder in map_folders]
    max_workers = 12
    items_in_queue = max_workers * 3
    queued_maps = items_in_queue
    cancel = False
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        map_tasks = [(executor.submit(full_load_map, map_folder), map_folder) for map_folder in map_folders[:items_in_queue]]
        while len(map_tasks) > 0:
            map_task, map_folder = map_tasks.pop(0)
            try:
                if cancel:
                    map_task.cancel()
                    continue
                results = map_task.result()
                for result in results:
                    x_context_prev_audio, x_context_prev_notes, x_context_audio, y_context_notes, z_timing_counts, z_note_counts, z_note_pos_counts, z_acc_prediction, z_speed_prediction = result
                    yield (x_context_prev_audio), (x_context_prev_notes), (x_context_audio), z_timing_counts, z_note_counts/20, z_note_pos_counts/10, z_acc_prediction, z_speed_prediction, (y_context_notes)
            except InterruptedError as ke:
                cancel = True
            except Exception as exc:
                pass
                # if str(exc) != "'_version'" and str(exc) != 'not v2':
                #     print(map_folder)
                #     print(exc)
                #     traceback.print_exc()
            finally:
                if not cancel:
                    queued_maps += 1
                    if queued_maps < len(map_folders):
                        map_tasks.append((executor.submit(full_load_map, map_folders[queued_maps]), map_folders[queued_maps]))

In [5]:
def create_ds_for_files(map_folders, batch_size, name, cache=False, shuffle=False):
    ds = tf.data.Dataset.from_generator(data_generator_multi_process, args=[map_folders], output_signature=(
        tf.TensorSpec(shape=(None, 2, 87, 129), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2, 40, 25), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2, 87, 129), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.int32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 2, 40, 25), dtype=tf.float32),
        # tf.TensorSpec(shape=(None, 1025, 44), dtype=tf.float32),
        # tf.TensorSpec(shape=(None, 35), dtype=tf.float32),
    ))
    ds = ds.flat_map(lambda x1, x2, x3, x4, x5, x6, x7, x8, y: tf.data.Dataset.from_tensor_slices((x1, x2, x3, x4, x5, x6, x7, x8, y)))
    # ds = ds.prefetch(20000)

    if cache:
        # ds = ds.cache()
        ds = ds.cache(f"./somethingsomething/{name}")
    if shuffle:
        ds = ds.shuffle(5000, reshuffle_each_iteration=True)
        # ds = ds.shuffle(len([v for v in ds]), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(256)
    return ds

In [6]:
# I currently cache the entire dataset, since the data loading part is quite compute intensive. Added a limit of 50 maps to avoid running out of ram on a test run.
maps = [path.replace("\\", "/") for path in glob.glob("../data/maps/*")]
random.shuffle(maps)
# maps = maps[:50]

In [7]:
batch_size = 16
val_split = 0.1
train_ds = create_ds_for_files(maps[int(len(maps)*val_split):], batch_size, "train", False, True)
val_ds = create_ds_for_files(maps[:int(len(maps)*val_split)], batch_size, "val", False, False)

In [8]:
# # preload the dataset into cache to keep the data loading errors away from the training logs
# discard_value = [0 for v in tqdm(train_ds)]

# discard_value = [0 for v in tqdm(val_ds)]

In [9]:
def audio_block():
    input_audio = tf.keras.Input(shape=(87, 129, 1), dtype="float32")
    l = input_audio
    l = tf.keras.layers.Conv2D(128, 5, activation="relu", padding="same")(l)
    l = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="same")(l)
    l = tf.keras.layers.Conv2D(128, 3, activation="relu")(l)
    l = tf.keras.layers.MaxPooling2D(pool_size=(1, 2))(l)
    l = tf.keras.layers.Conv2D(128, 3, activation="relu")(l)
    l = tf.keras.layers.MaxPooling2D(pool_size=(1, 2))(l)
    l = tf.keras.layers.Reshape((40, -1))(l)
    l = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation="tanh"))(l)
    return tf.keras.Model(input_audio, l)

In [10]:
def audio_block_stereo():
    l_audio_block = audio_block()
    
    input_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    l = input_audio
    l = tf.keras.layers.TimeDistributed(l_audio_block)(l)
    l = tf.keras.layers.Permute((2, 1, 3))(l)
    l = tf.keras.layers.Reshape((40, -1))(l)
    l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(l)
    l = tf.keras.layers.LSTM(64, return_sequences=True)(l)
    return tf.keras.Model(input_audio, l)

In [11]:
def note_positioning_block():
    input_timings = tf.keras.Input(shape=(2, 40, 1), dtype="float32")
    input_features = tf.keras.Input(shape=(40, 128), dtype="float32")
    
    l_timings = tf.keras.layers.Permute((2, 1, 3))(input_timings)
    l_timings = tf.keras.layers.Reshape((40, -1))(l_timings)

    l = tf.keras.layers.Concatenate(axis=2)([l_timings, input_features])
    l = tf.keras.layers.LSTM(256, return_sequences=True)(l)
    l = tf.keras.layers.LSTM(256, return_sequences=True)(l)
    l_pos_out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(24, activation="sigmoid"))(l)
    l = tf.keras.layers.Concatenate(axis=2)([l_pos_out, l])
    l = tf.keras.layers.LSTM(128, return_sequences=True)(l)
    l = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation="relu"))(l)
    l_angle_out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(24, activation="linear"))(l)
    
    l_pos_out = tf.keras.layers.Reshape((40, 2, -1))(l_pos_out)
    l_pos_out = tf.keras.layers.Permute((2, 1, 3))(l_pos_out)
    l_angle_out = tf.keras.layers.Reshape((40, 2, -1))(l_angle_out)
    l_angle_out = tf.keras.layers.Permute((2, 1, 3))(l_angle_out)
    
    return tf.keras.Model([input_timings, input_features], [l_pos_out, l_angle_out])

In [12]:
def make_model(audio_model):
    input_prev_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    input_prev_notes = tf.keras.Input(shape=(2, 40, 25), dtype="float32")
    input_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    input_acc_prediction = tf.keras.Input(shape=(1), dtype="float32")
    input_speed_prediction = tf.keras.Input(shape=(1), dtype="float32")

    l_prev_audio = audio_model(input_prev_audio)
    l_audio = audio_model(input_audio)
    
    l_prev_notes = tf.keras.layers.Permute((2, 1, 3))(input_prev_notes)
    l_prev_notes = tf.keras.layers.Reshape((40, -1))(l_prev_notes)
    
    l_prev = tf.keras.layers.Concatenate(axis=2)([l_prev_audio, l_prev_notes])
    l_prev = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(l_prev)
    l_prev = tf.keras.layers.LSTM(64)(l_prev)
    
    l_input_acc_prediction = tf.keras.layers.RepeatVector(40)(input_acc_prediction)
    l_input_speed_prediction = tf.keras.layers.RepeatVector(40)(input_speed_prediction)
    l_prev = tf.keras.layers.RepeatVector(40)(l_prev)
    l = tf.keras.layers.Concatenate(axis=2)([l_audio, l_prev, l_input_acc_prediction, l_input_speed_prediction])
    l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(l)
    l = tf.keras.layers.LSTM(128, return_sequences=True)(l)
    l_timings_out = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2, activation="sigmoid"))(l)
    l_timings_out = tf.keras.layers.Permute((2, 1))(l_timings_out)
    l_timings_out = tf.keras.layers.Reshape((2, 40, -1))(l_timings_out)

    note_positioning_l = note_positioning_block()
    
    l_pos_out, l_angle_out = note_positioning_l([l_timings_out, l])

    model = tf.keras.Model(inputs = [input_prev_audio, input_prev_notes, input_audio, input_acc_prediction, input_speed_prediction], outputs = [l_timings_out, l_pos_out, l_angle_out])
    return model

In [13]:
def make_discriminator_model(audio_model):
    input_prev_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")
    input_audio = tf.keras.Input(shape=(2, 87, 129, 1), dtype="float32")

    input_prev_notes = tf.keras.Input(shape=(2, 40, 25), dtype="float32")
    input_notes = tf.keras.Input(shape=(2, 40, 25), dtype="float32")

    input_acc_prediction = tf.keras.Input(shape=(1), dtype="float32")
    input_speed_prediction = tf.keras.Input(shape=(1), dtype="float32")
    
    l_acc_prediction = tf.keras.layers.RepeatVector(80)(input_acc_prediction)
    l_speed_prediction = tf.keras.layers.RepeatVector(80)(input_speed_prediction)
    
    l_prev_audio = audio_model(input_prev_audio)
    l_audio = audio_model(input_audio)

    l_audio = tf.keras.layers.Concatenate(axis=1)([l_prev_audio, l_audio])
    l_notes = tf.keras.layers.Concatenate(axis=2)([input_prev_notes, input_notes])
    
    l_notes = tf.keras.layers.Permute((2, 1, 3))(l_notes)
    l_notes = tf.keras.layers.Reshape((80, -1))(l_notes)
    
    l = tf.keras.layers.Concatenate(axis=2)([l_audio, l_notes, l_acc_prediction, l_speed_prediction])
    
    l = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(l)
    l = tf.keras.layers.LSTM(256, return_sequences=True)(l)
    l = tf.keras.layers.LSTM(256)(l)
    l = tf.keras.layers.Dense(1, activation="sigmoid")(l)
    
    return tf.keras.Model([input_prev_audio, input_audio, input_prev_notes, input_notes, input_acc_prediction, input_speed_prediction], [l])

In [14]:
audio_model = audio_block_stereo()

model_generator = make_model(audio_model)
model_discriminator = make_discriminator_model(audio_model)
# model.summary()

In [15]:
# # visualize the model
# tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, expand_nested=True)

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [17]:
metric_train_d_loss = tf.keras.metrics.Mean(name='metric_train_d_loss')
metric_train_g_loss = tf.keras.metrics.Mean(name='metric_train_g_loss')
metric_train_simple_loss = tf.keras.metrics.Mean(name='metric_train_simple_loss')

metric_train_timing_loss = tf.keras.metrics.Mean(name='metric_train_timing_loss')
metric_train_pos_loss = tf.keras.metrics.Mean(name='metric_train_pos_loss')
metric_train_angle_loss = tf.keras.metrics.Mean(name='metric_train_angle_loss')
metric_train_loss = tf.keras.metrics.Mean(name='metric_train_loss')
metric_train_discriminator_loss = tf.keras.metrics.Mean(name='metric_train_discriminator_loss')
metric_val_timing_loss = tf.keras.metrics.Mean(name='metric_val_timing_loss')
metric_val_pos_loss = tf.keras.metrics.Mean(name='metric_val_pos_loss')
metric_val_angle_loss = tf.keras.metrics.Mean(name='metric_val_angle_loss')
metric_val_loss = tf.keras.metrics.Mean(name='metric_val_loss')
metric_val_discriminator_loss = tf.keras.metrics.Mean(name='metric_val_discriminator_loss')

### Custom loss
Calculates 2 separate values:
 - timing loss - simple loss based on when the notes were placed
 - positioning loss - loss based on the position and direction of the placed note, adjusted for the number of notes that appear in different positions to avoid a massive bias towards placing most commonly appearing notes

In [18]:
tf.config.run_functions_eagerly(False)

In [19]:
@tf.function
def custom_loss(y, predictions):
    timing_predictions, pos_predictions, angle_predictions = predictions
    org_timing_loss_matrix = tf.square(y[:, :, :, :1] - timing_predictions) * (y[:, :, :, :1] + 0.08)
    timing_loss_matrix = org_timing_loss_matrix
    timing_loss = tf.reduce_mean(timing_loss_matrix) * 10
    
    # positioning_loss_matrix = tf.square(y[:, :, 1:] - positioning_predictions) * (y[:, :, :1]) * (y[:, :, 1:] * note_poss_loss_balance + 0.0169)
    # positioning_loss = tf.reduce_sum(positioning_loss_matrix) / tf.reduce_sum(y[:, :, 1:]) * 0.5
    
    y_positioning_loss_matrix = tf.square(y[:, :, :, 1::2] - pos_predictions) * (y[:, :, :, :1]) * (y[:, :, :, 1::2] + 0.069)
    y_positioning_loss = tf.reduce_sum(y_positioning_loss_matrix) / tf.reduce_sum(y[:, :, :, 1::2]) * 0.5
    
    y_positioning_angle_loss_matrix = tf.square(tf.minimum(tf.abs(y[:, :, :, 2::2] - angle_predictions), tf.minimum(tf.abs(y[:, :, :, 2::2] - angle_predictions - 1), tf.abs(y[:, :, :, 2::2] - angle_predictions + 1)))) * (y[:, :, :, 1::2])
    y_positioning_angle_loss = tf.reduce_sum(y_positioning_angle_loss_matrix) / tf.reduce_sum(y[:, :, :, 1::2]) * 20
    
    loss = timing_loss + y_positioning_loss + y_positioning_angle_loss

    return timing_loss, y_positioning_loss, y_positioning_angle_loss, loss

In [20]:
@tf.function
def train_step(optimizer, data):
    x1, x2, x3, x4, x5, x6, x7, x8, y = data
    timing_predictions, pos_predictions, angle_predictions = model_generator([x1, x2, x3, x7, x8], training=True)
    
    generator_output = tf.concat([timing_predictions, pos_predictions, angle_predictions], axis=3)
    reshaped_x2 = tf.concat([x2[:, :, :, :1], x2[:, :, :, 1::2], x2[:, :, :, 2::2]], axis=3)
    reshaped_y = tf.concat([y[:, :, :, :1], y[:, :, :, 1::2], y[:, :, :, 2::2]], axis=3)
    labels = tf.concat([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0)
    labels += 0.05 * tf.random.uniform(labels.shape)
    
    with tf.GradientTape() as tape:
        predictions = model_discriminator([tf.concat([x1, x1], axis=0), tf.concat([x3, x3], axis=0), tf.concat([reshaped_x2, reshaped_x2], axis=0), tf.concat([generator_output, reshaped_y], axis=0), tf.concat([x7, x7], axis=0), tf.concat([x8, x8], axis=0)])
        d_loss = loss_fn(labels, predictions)

    grads = tape.gradient(d_loss, model_discriminator.trainable_weights)
    discriminator_optimizer.apply_gradients(zip(grads, model_discriminator.trainable_weights))
    
    misleading_labels = tf.zeros((batch_size, 1))
    with tf.GradientTape() as tape:
        timing_predictions, pos_predictions, angle_predictions = model_generator([x1, x2, x3, x7, x8], training=True)
        generator_output = tf.concat([timing_predictions, pos_predictions, angle_predictions], axis=3)
        predictions = model_discriminator([x1, x3, reshaped_x2, generator_output, x7, x8])
        g_loss = loss_fn(misleading_labels, predictions)
        
    grads = tape.gradient(g_loss, model_generator.trainable_weights)
    generator_optimizer.apply_gradients(zip(grads, model_generator.trainable_weights))
    
    with tf.GradientTape() as tape:
        timing_predictions, pos_predictions, angle_predictions = model_generator([x1, x2, x3, x7, x8], training=True)
        timing_loss, y_positioning_loss, y_positioning_angle_loss, simple_loss = custom_loss(y, (timing_predictions, pos_predictions, angle_predictions))
        
    grads_simple = tape.gradient(simple_loss, model_generator.trainable_weights)
    optimizer.apply_gradients(zip(grads_simple, model_generator.trainable_weights))
    
    metric_train_d_loss(d_loss)
    metric_train_g_loss(g_loss)
    metric_train_simple_loss(simple_loss)

# @tf.function
# def val_step(data):
#     x1, x2, x3, x4, x5, x6, x7, x8, y = data
    
#     predictions = model([x1, x2, x3, x7, x8], training=False)
#     # with tf.device('/CPU:0'):
#     timing_loss, positioning_loss, angle_loss, loss = custom_loss(y, predictions)
        
#     metric_train_d_loss(d_loss)
#     metric_train_g_loss(g_loss)

### Results validation

- specify the correct folder with maps for which you would want to generate the map
- add maps that you want to use for testing, better to avoid using the maps that already exist in the training dataset to avoid false positives of AI learning a specific map
- add an Expert diff if doesn't exist, currently hardcoded to just override the Expert diff to avoid setting up all the metadata

In [21]:
def full_validation(timing_threshhold, epoch, acc_prediction, speed_prediction):
    base_validation_path = "./validation"
    os.makedirs(base_validation_path, exist_ok=True)
    
    validation_map = "DA42AF71F4CA5AD280C3F69BCA0BD6C6D1CDA06E"
    validation_map_dir = f"{base_validation_path}/{validation_map}"
    (song_data, segment_duration), diffs = load_map(validation_map_dir)
    
    
    angle_to_direction = {
        180:0,
        0:1,
        90:2,
        270:3,
        135:4,
        225:5,
        45:6,
        315:7
    }
    direction_to_angle = {
        0: 180,
        1: 0,
        2: 90,
        3: 270,
        4: 135,
        5: 225,
        6: 45,
        7: 315
    }


    def get_note_angle(direction):
        return direction_to_angle[direction] / 360
    
    def get_note_direction(angle):
        angle = int(angle * 360)
        if angle % 45 > 22.5:
            angle += 45
        angle = (angle - angle % 45) % 360
        return angle_to_direction[angle]

    def validate_model(song_data, segment_duration, timing_threshhold, positioning_threshhold, intensity_1, intensity_2, acc_prediction, speed_prediction, note_pos_count):
        context_length = 1
        prediction_note_count = context_length * 40
        prediction_note_time_length = context_length / prediction_note_count

        context_steps = int(context_length / segment_duration) + 1
        step_size = context_steps
        
        generated_notes = []
        max_val_timing = 0
        max_val_positioning = 0
        
        zero_notes = 1
        one_notes = 1
        
        prev_note_segment = ([[0]*25 for i in range(prediction_note_count)], [[0]*25 for i in range(prediction_note_count)])
        prev_audio_segment = song_data[:, :context_steps, :]
        with tqdm(range(context_steps, song_data.shape[1] - context_steps, step_size)) as _tqdm:
          for i in _tqdm:
            curr_time = i * segment_duration
            
            x_context_prev_audio = prev_audio_segment
            x_context_prev_notes = prev_note_segment
            x_context_audio = song_data[:, i:i+context_steps, :]
            timing_prediction, placement_prediction, placement_angle_prediction = model_generator([np.array([x_context_prev_audio]), np.array([x_context_prev_notes]), np.array([x_context_audio]), np.array([acc_prediction + (random.random() - 0.5) * 0.1]), np.array([speed_prediction + (random.random() - 0.5) * 0.1])], training=False)
            timing_prediction = tf.where(timing_prediction > timing_threshhold, 1, 0)
            timing_prediction = np.array(timing_prediction[0])
            placement_prediction = np.array(placement_prediction[0])
            placement_angle_prediction = np.array(placement_angle_prediction[0])
            
            
            x_context_prev_audio = x_context_audio
            prev_note_segment = ([[0]*25 for i in range(prediction_note_count)], [[0]*25 for i in range(prediction_note_count)])

            # I use them to find values that would generate a reasonable number of notes.
            # Small adjustments to the model and it's loss function can significantly shift the actual number values.
            # if max_val_timing < np.max(timing_prediction):
            #     max_val_timing = np.max(timing_prediction)
            #     print(f"max_timing: {max_val_timing}")
                
                # if max_val_positioning < np.max(positioning_prediction):
                #     max_val_positioning = np.max(positioning_prediction)
                #     print(f"max_positioning: {max_val_positioning}")
            
            for j in range(prediction_note_count):
                for color in range(2):
                    curr_note_time = curr_time + j * prediction_note_time_length
                    prediction_timing = timing_prediction[color][j][0]
                    if prediction_timing < timing_threshhold:
                        continue
                    prediction_positioning = placement_prediction[color][j]
                    prediction_angle = placement_angle_prediction[color][j]
                    
                    # prediction_positioning[:12] = prediction_positioning[:12] * (one_notes / (zero_notes + one_notes + 0.00001))
                    # prediction_positioning[12:] = prediction_positioning[12:] * (zero_notes / (zero_notes + one_notes + 0.00001))

                    # Place only 1 note per timing or to place many notes. More than 1 note can get super repetitive more easily, but both are of quite poor quality so far.
                    max_one_note_per_placement = True
                    if max_one_note_per_placement:
                        note_prediction_iter = np.argmax(prediction_positioning)
                        prediction_positioning_enumerated = [(note_prediction_iter, prediction_positioning[note_prediction_iter], prediction_angle[note_prediction_iter])]
                    else:
                        prediction_positioning_enumerated = [(i, note_prediction, prediction_angle[i]) for i, note_prediction in enumerate(prediction_positioning) if note_prediction > positioning_threshhold]

                    for note_prediction_iter, note_prediction, angle_prediction in prediction_positioning_enumerated:
                            line_layer = note_prediction_iter % 3
                            line_index = int(note_prediction_iter / 3) % 4
                            direction = get_note_direction(angle_prediction)
                            generated_notes.append(Note(curr_note_time, line_index, line_layer, color, direction))
                            if color == 0:
                                zero_notes += 1
                            else:
                                one_notes += 1
                            prev_note_segment[color][j][0] = 1
                            prev_note_segment[color][j][1 + note_prediction_iter * 2] = 1
                            prev_note_segment[color][j][1 + note_prediction_iter * 2 + 1] = get_note_angle(direction)
            if len(generated_notes) > 0:
                average_notes_per_second = len(generated_notes)/generated_notes[-1].time
            else:
                average_notes_per_second = -1
            _tqdm.set_postfix(average_notes_per_second=average_notes_per_second)
        
        generated_notes.sort(key=lambda note: note.time)
        return generated_notes

    intensity_timings_per_second = 7 # model input for number of correct timings per second
    intensity_notes_per_second = intensity_timings_per_second # model input for sum of '1's in the prediction segment. Increasing this value should result in more stacks and sliders.
    note_pos_count = 3
    generated_notes = validate_model(song_data, segment_duration, timing_threshhold=timing_threshhold, positioning_threshhold=0.45, intensity_1=intensity_timings_per_second, intensity_2=intensity_notes_per_second/20, acc_prediction=acc_prediction, speed_prediction=speed_prediction, note_pos_count=note_pos_count/10)

    if len(generated_notes) > 0:
        average_notes_per_second = len(generated_notes)/generated_notes[-1].time
    else:
        average_notes_per_second = -1
    
    with open(validation_map_dir + "/Info.dat", "rb") as f:
        info_json = json.load(f)
        bpm = info_json["_beatsPerMinute"]
        
    with open(validation_map_dir + "/ExpertStandard.dat", "rb") as f:
        diff_json = json.load(f)

    diff_json["_notes"] = [{"_time": note.time / 60 * bpm, "_lineIndex": int(note.lineIndex), "_lineLayer": int(note.lineLayer), "_type": int(note.type), "_cutDirection": int(note.direction)} for note in generated_notes]
    if len(diff_json["_notes"]) == 0:
        diff_json["_notes"] = [{"_time": 1, "_lineIndex": 0, "_lineLayer": 0, "_cutDirection": 0, "_type": 0}]
    with open(validation_map_dir + "/ExpertStandard.dat", "w") as f:
        json.dump(diff_json, f)
        
    shutil.make_archive(f"{validation_map_dir}q{epoch}q{timing_threshhold}q{average_notes_per_second}q{acc_prediction}q{speed_prediction}", 'zip', validation_map_dir)

In [22]:
for epoch in range(0, 15):
    metric_train_d_loss.reset_states()
    metric_train_g_loss.reset_states()
    metric_train_timing_loss.reset_states()
    metric_train_pos_loss.reset_states()
    metric_train_angle_loss.reset_states()
    metric_train_loss.reset_states()
    metric_val_timing_loss.reset_states()
    metric_val_pos_loss.reset_states()
    metric_val_angle_loss.reset_states()
    metric_val_loss.reset_states()

    if epoch <= 0:
        generator_optimizer.learning_rate.assign(0.0001)
        discriminator_optimizer.learning_rate.assign(0.00002)
        optimizer.learning_rate.assign(0.00002)
    elif epoch <= 2:
        generator_optimizer.learning_rate.assign(0.00005)
        discriminator_optimizer.learning_rate.assign(0.00001)
        optimizer.learning_rate.assign(0.00001)
    elif epoch <= 4:
        generator_optimizer.learning_rate.assign(0.000025)
        discriminator_optimizer.learning_rate.assign(0.000005)
        optimizer.learning_rate.assign(0.000005)
    elif epoch <= 6:
        generator_optimizer.learning_rate.assign(0.000025)
        discriminator_optimizer.learning_rate.assign(0.0000025)
        optimizer.learning_rate.assign(0.0000025)
    elif epoch <= 8:
        generator_optimizer.learning_rate.assign(0.00001)
        discriminator_optimizer.learning_rate.assign(0.00001)
        optimizer.learning_rate.assign(0.00001)
    elif epoch <= 10:
        generator_optimizer.learning_rate.assign(0.0000025)
        discriminator_optimizer.learning_rate.assign(0.0000025)
        optimizer.learning_rate.assign(0.0000025)
    
    with tqdm(train_ds.enumerate(), unit="batch") as _tqdm:
        _tqdm.set_description(f"Epoch train: {epoch}")
        for step, data in _tqdm:
            train_step(optimizer, data)
            _tqdm.set_postfix(
                d_loss=metric_train_d_loss.result().numpy(),
                g_loss=metric_train_g_loss.result().numpy(),
                simple_loss=metric_train_simple_loss.result().numpy(),
            )
    
    # if 'val_ds' in locals() or 'val_ds' in globals():
    #     with tqdm(val_ds.enumerate(), unit="batch") as _tqdm:
    #         _tqdm.set_description(f"Epoch val: {epoch}")
    #         for step, data in _tqdm:
    #             val_step(model, data)
    #             _tqdm.set_postfix(
    #                 timing_loss=metric_val_timing_loss.result().numpy(),
    #                 pos_loss=metric_val_pos_loss.result().numpy(),
    #                 angle_loss=metric_val_angle_loss.result().numpy(),
    #                 loss=metric_val_loss.result().numpy(),
    #             )
    
    full_validation(0.825 + (random.random() - 0.5)*0.2, epoch, 0.775 + (random.random() - 0.5)*0.25, 0.35 + (random.random() - 0.5)*0.25)
    full_validation(0.825 + (random.random() - 0.5)*0.2, epoch, 0.775 + (random.random() - 0.5)*0.25, 0.35 + (random.random() - 0.5)*0.25)


Epoch train: 0: : 0batch [00:00, ?batch/s]

Epoch train: 0: : 13096batch [32:50,  7.20batch/s, d_loss=0.661, g_loss=0.912, simple_loss=2.79]

In [ ]:
full_validation(0.1 + (random.random() - 0.5)*0.1, epoch, 0.8 + (random.random() - 0.5)*0.15, 0.3 + (random.random() - 0.5)*0.4)

100%|██████████| 169/169 [00:17<00:00,  9.40it/s, average_notes_per_second=1.98]
